# Segmenting and Clustering Neighborhoods in Toronto

Importing libraries

In [1]:
import pandas as pd # library for data analysis
import requests
import folium
import os
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup #for scraping
from sklearn.cluster import KMeans

%matplotlib inline
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


<a id='menu1'></a>

## Part 1

We are going to scraping the neighborhood data from Wikipedia. 

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(wiki)

In [3]:
soup = BeautifulSoup(r.content)
trs = soup.find('table','wikitable sortable').find_all('tr')
columns = ['postcode', 'borough', 'neighborhood']
df_postcode = pd.DataFrame(columns=columns)
for tr in trs:
    tds = tr.find_all('td')
    if len(tds) == 0: continue
    postcode = tds[0].get_text().strip()
    borough = tds[1].get_text().strip()
    neighborhood = tds[2].get_text().strip()
    if borough != 'Not assigned':
#         print("{} - {} - {}".format(postcode, borough, neighborhood))
        df_postcode = df_postcode.append({'postcode': postcode, 'borough': borough, 'neighborhood':neighborhood}, ignore_index=True)

print(df_postcode.shape[0])
df_postcode.head()

211


postcode           borough      neighborhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights

Find ___"Not assigned"___ neighborhood and replace it with borough value

In [4]:
df_postcode[df_postcode.neighborhood=='Not assigned']

postcode       borough  neighborhood
6      M7A  Queen's Park  Not assigned

In [5]:
df_postcode['neighborhood'] = df_postcode.apply(lambda x: x.borough if x.neighborhood == 'Not assigned' else x.neighborhood, axis=1)

In [6]:
df_postcode[df_postcode.neighborhood=='Not assigned']

Empty DataFrame
Columns: [postcode, borough, neighborhood]
Index: []

Now there is no neighborhood with ___"Not Assigned"___ value

Next step is __grouping neighborhood__ with the same postcode into one row

In [7]:
df_postcode_grouped = df_postcode.groupby(['postcode','borough'], as_index=False).count()
df_postcode_grouped
df_postcode_clean = pd.DataFrame(columns=columns)
for index, row in df_postcode_grouped.iterrows():
    neighborhood = df_postcode[df_postcode.postcode==row[0]].neighborhood.str.cat(sep=',')
    df_postcode_clean = df_postcode_clean.append({'postcode':row[0], 'borough':row[1], 'neighborhood':neighborhood}, ignore_index=True)
    
print(df_postcode_clean.shape)
df_postcode_clean.head()

(103, 3)


postcode      borough                          neighborhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae

In [8]:
df_postcode_clean.shape

(103, 3)

## Part 2

Find __coordinate__ of each postcode by reading from https://cocl.us/Geospatial_data

In [12]:
# # read from url
try:
    url = 'https://cocl.us/Geospatial_data'
    df_coordinate = pd.read_csv(url)
except:
    df_coordinate = pd.read_csv('data/coordinate.csv')

Postcode' coordinates dataframe:

In [14]:
df_coordinate.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

__Join__ with df_postcode_clean dataframe

In [16]:
df_postcode_coordinate = df_postcode_clean.join(df_coordinate.set_index('Postal Code'), on='postcode')
df_postcode_coordinate.head()

postcode      borough                          neighborhood   Latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [17]:
df_postcode_coordinate.shape

(103, 5)

__All__ postcode now has coordinates

## Part 3

In [18]:
map_city = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

for lat, long, postcode, borough, neighborhood in zip(df_postcode_coordinate.Latitude, df_postcode_coordinate.Longitude, df_postcode_coordinate.postcode, df_postcode_coordinate.borough, df_postcode_coordinate.neighborhood):
        folium.CircleMarker(
            [lat, long],
            radius=5,
            color='yellow',
            fill=True,
            popup="<b>{}</b>\n{}:\n{}".format(postcode, borough, neighborhood),
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(map_city)

map_city

In [19]:
CLIENT_ID = 'WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG' # your Foursquare ID
CLIENT_SECRET = 'XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG
CLIENT_SECRET:XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY


#### Let's explore the first neighborhood in our dataframe.

In [20]:
neighborhood_latitude, neighborhood_longitude = df_postcode_coordinate.iloc[0].Latitude, df_postcode_coordinate.iloc[0].Longitude
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v=20180605&ll={},{}&radius=500&limit=100".format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG&client_secret=XSKLTD0Y2MCLUS4DT105KOZ1KBD3GAEGVG4RDR2ZHZBT5NWY&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100


In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd100151ed2196c3d355f2a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories        lat        lng
0  Wendy's  Fast Food Restaurant  43.807448 -79.199056

In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## 2. Explore Neighborhoods in City of Toronto

Again borrowing funtion from the course

In [25]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# district_venues_csv = 'data\district_venues.csv' 
# if not os.path.exists(district_venues_csv):
district_venues = getNearbyVenues(names=df_postcode_coordinate['postcode'], latitudes=df_postcode_coordinate['Latitude'],longitudes=df_postcode_coordinate['Longitude'],radius=500)
# else:
#     district_venues = pd.read_csv(district_venues_csv)

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [27]:
district_venues.shape[0]

2243

In [28]:
district_venues.head()

Postcode  Postcode Latitude  Postcode Longitude  \
0      M1B          43.806686          -79.194353   
1      M1C          43.784535          -79.160497   
2      M1E          43.763573          -79.188711   
3      M1E          43.763573          -79.188711   
4      M1E          43.763573          -79.188711   

                             Venue  Venue Latitude  Venue Longitude  \
0                          Wendy's       43.807448       -79.199056   
1            Royal Canadian Legion       43.782533       -79.163085   
2  Swiss Chalet Rotisserie & Grill       43.767697       -79.189914   
3                G & G Electronics       43.765309       -79.191537   
4                       Marina Spa       43.766000       -79.191000   

         Venue Category  
0  Fast Food Restaurant  
1                   Bar  
2           Pizza Place  
3     Electronics Store  
4                   Spa

Let's check how many venues were returned for each neighborhood

In [31]:
district_venues.groupby('Postcode', as_index=False).count()

Postcode  Postcode Latitude  Postcode Longitude  Venue  Venue Latitude  \
0       M1B                  1                   1      1               1   
1       M1C                  1                   1      1               1   
2       M1E                  8                   8      8               8   
3       M1G                  4                   4      4               4   
4       M1H                  7                   7      7               7   
5       M1J                  2                   2      2               2   
6       M1K                  4                   4      4               4   
7       M1L                  9                   9      9               9   
8       M1M                  2                   2      2               2   
9       M1N                  4                   4      4               4   
10      M1P                  6                   6      6               6   
11      M1R                  7                   7      7               7   
12      M1S                  4                   4      4               4   
13      M1T                 10                  10     10              10   
14      M1V                  3                   3      3               3   
15      M1W                 13                  13     13              13   
16      M2H                  6                   6      6               6   
17      M2J                 66                  66     66              66   
18      M2K                  4                   4      4               4   
19      M2L                  2                   2      2               2   
20      M2N                 34                  34     34              34   
21      M2P                  3                   3      3               3   
22      M2R                  5                   5      5               5   
23      M3A                  4                   4      4               4   
24      M3B                  4                   4      4               4   
25      M3C                 21                  21     21              21   
26      M3H                 19                  19     19              19   
27      M3J                  5                   5      5               5   
28      M3K                  3                   3      3               3   
29      M3L                  4                   4      4               4   
..      ...                ...                 ...    ...             ...   
70      M6B                  4                   4      4               4   
71      M6C                  3                   3      3               3   
72      M6E                  6                   6      6               6   
73      M6G                 16                  16     16              16   
74      M6H                 20                  20     20              20   
75      M6J                 63                  63     63              63   
76      M6K                 19                  19     19              19   
77      M6L                  4                   4      4               4   
78      M6M                  4                   4      4               4   
79      M6N                  4                   4      4               4   
80      M6P                 23                  23     23              23   
81      M6R                 15                  15     15              15   
82      M6S                 40                  40     40              40   
83      M7A                 39                  39     39              39   
84      M7R                 10                  10     10              10   
85      M7Y                 17                  17     17              17   
86      M8V                 15                  15     15              15   
87      M8W                 10                  10     10              10   
88      M8X                  2                   2      2               2   
89      M8Y                  1                   1      1               1   

#### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(district_venues['Venue Category'].unique())))

There are 274 uniques categories.


<a id='menu3'></a>

## 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
district_onehot = pd.get_dummies(district_venues[['Venue Category']], prefix="", prefix_sep="")

#Add Neighborhood name
district_onehot = pd.concat([district_venues[['Postcode']], district_onehot], axis=1)

district_onehot.head(10)

Postcode  Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0      M1B                  0               0                  0        0   
1      M1C                  0               0                  0        0   
2      M1E                  0               0                  0        0   
3      M1E                  0               0                  0        0   
4      M1E                  0               0                  0        0   
5      M1E                  0               0                  0        0   
6      M1E                  0               0                  0        0   
7      M1E                  0               0                  0        0   
8      M1E                  0               0                  0        0   
9      M1E                  0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   
5                   0             0               0                0   
6                   0             0               0                0   
7                   0             0               0                0   
8                   0             0               0                0   
9                   0             0               0                0   

   Airport Terminal     ...       Train Station  \
0                 0     ...                   0   
1                 0     ...                   0   
2                 0     ...                   0   
3                 0     ...                   0   
4                 0     ...                   0   
5                 0     ...                   0   
6                 0     ...                   0   
7                 0     ...                   0   
8                 0     ...                   0   
9                 0     ...                   0   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   
5                              0                 0            0   
6                              0                 0            0   
7                              0                 0            0   
8                              0                 0            0   
9                              0                 0            0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
0                      0                0         0            0   
1                      0                0         0            0   
2                      0                0         0            0   
3                      0                0         0            0   
4                      0                0         0            0   
5                      0                0         0            0   
6                      0                0         0            0   
7                      0                0         0            0   
8                      0                0         0            0   
9                      0                0         0            0   

   Women's Store  Yoga Studio  
0              0            0  
1              0            0  
2              0            0  
3              0            0  
4              0            0  
5              0            0  
6              0            0  
7              0            0  
8              0        

And let's examine the new dataframe size.

In [34]:
district_onehot.shape

(2243, 275)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
district_grouped = district_onehot.groupby('Postcode').mean().reset_index()
district_grouped.head()

Postcode  Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0      M1B                0.0             0.0                0.0      0.0   
1      M1C                0.0             0.0                0.0      0.0   
2      M1E                0.0             0.0                0.0      0.0   
3      M1G                0.0             0.0                0.0      0.0   
4      M1H                0.0             0.0                0.0      0.0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                 0.0           0.0             0.0              0.0   
1                 0.0           0.0             0.0              0.0   
2                 0.0           0.0             0.0              0.0   
3                 0.0           0.0             0.0              0.0   
4                 0.0           0.0             0.0              0.0   

   Airport Terminal     ...       Train Station  \
0               0.0     ...                 0.0   
1               0.0     ...                 0.0   
2               0.0     ...                 0.0   
3               0.0     ...                 0.0   
4               0.0     ...                 0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                            0.0               0.0          0.0   
1                            0.0               0.0          0.0   
2                            0.0               0.0          0.0   
3                            0.0               0.0          0.0   
4                            0.0               0.0          0.0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
0                    0.0              0.0       0.0          0.0   
1                    0.0              0.0       0.0          0.0   
2                    0.0              0.0       0.0          0.0   
3                    0.0              0.0       0.0          0.0   
4                    0.0              0.0       0.0          0.0   

   Women's Store  Yoga Studio  
0            0.0          0.0  
1            0.0          0.0  
2            0.0          0.0  
3            0.0          0.0  
4            0.0          0.0  

[5 rows x 275 columns]

#### Let's confirm the new size

In [36]:
district_grouped.shape

(100, 275)

#### Let's print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in district_grouped['Postcode']:
    print("----"+hood+"----")
    temp = district_grouped[district_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True)[:5])
    print('\n')

----M1B----
                             venue  freq
0             Fast Food Restaurant   1.0
1        Middle Eastern Restaurant   0.0
2                            Motel   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----M1C----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2                            Motel   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----M1E----
                 venue  freq
0                  Spa  0.12
1         Intersection  0.12
2  Rental Car Location  0.12
3    Electronics Store  0.12
4   Mexican Restaurant  0.12


----M1G----
                        venue  freq
0                 Coffee Shop  0.50
1            Insurance Office  0.25
2           Korean Restaurant  0.25
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


----M1H----
                  venue  freq
0  Caribbean Restaur

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = district_grouped['Postcode']

for ind in np.arange(district_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(district_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Postcode 1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0      M1B  Fast Food Restaurant             Drugstore    Dim Sum Restaurant   
1      M1C                   Bar   Dumpling Restaurant                 Diner   
2      M1E        Medical Center        Breakfast Spot   Rental Car Location   
3      M1G           Coffee Shop     Korean Restaurant      Insurance Office   
4      M1H    Athletics & Sports   Fried Chicken Joint      Hakka Restaurant   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0                 Diner        Discount Store               Dog Run   
1        Discount Store               Dog Run      Doner Restaurant   
2    Mexican Restaurant          Intersection           Pizza Place   
3             Drugstore    Dim Sum Restaurant                 Diner   
4       Thai Restaurant                Bakery  Caribbean Restaurant   

  7th Most Common Venue 8th Most Common Venue        9th Most Common Venue  \
0      Doner Restaurant            Donut Shop          Dumpling Restaurant   
1            Donut Shop             Drugstore                  Yoga Studio   
2                   Spa     Electronics Store  Eastern European Restaurant   
3        Discount Store               Dog Run             Doner Restaurant   
4                  Bank      Doner Restaurant                        Diner   

  10th Most Common Venue  
0            College Gym  
1           Dessert Shop  
2    Dumpling Restaurant  
3             Donut Shop  
4         Discount Store

<a id='menu4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [49]:
district_grouped.shape

(100, 275)

In [50]:
# set number of clusters
kclusters = 5

district_grouped_clustering = district_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(district_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [52]:
neighborhoods_venues_sorted.head()

Cluster Labels Postcode 1st Most Common Venue 2nd Most Common Venue  \
0               0      M1B  Fast Food Restaurant             Drugstore   
1               2      M1C                   Bar   Dumpling Restaurant   
2               2      M1E        Medical Center        Breakfast Spot   
3               2      M1G           Coffee Shop     Korean Restaurant   
4               2      M1H    Athletics & Sports   Fried Chicken Joint   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0    Dim Sum Restaurant                 Diner        Discount Store   
1                 Diner        Discount Store               Dog Run   
2   Rental Car Location    Mexican Restaurant          Intersection   
3      Insurance Office             Drugstore    Dim Sum Restaurant   
4      Hakka Restaurant       Thai Restaurant                Bakery   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0               Dog Run      Doner Restaurant            Donut Shop   
1      Doner Restaurant            Donut Shop             Drugstore   
2           Pizza Place                   Spa     Electronics Store   
3                 Diner        Discount Store               Dog Run   
4  Caribbean Restaurant                  Bank      Doner Restaurant   

         9th Most Common Venue 10th Most Common Venue  
0          Dumpling Restaurant            College Gym  
1                  Yoga Studio           Dessert Shop  
2  Eastern European Restaurant    Dumpling Restaurant  
3             Doner Restaurant             Donut Shop  
4                        Diner         Discount Store

In [80]:
district_merged = neighborhoods_venues_sorted.join(df_postcode_coordinate.set_index('postcode'), on='Postcode')

In [81]:
cols = district_merged.columns.tolist()
cols = cols[-4:]+[cols[1]]+[cols[0]]+cols[2:-4]
district_merged = district_merged[cols]

district_merged.head() # check the last columns!

borough                          neighborhood   Latitude  Longitude  \
0  Scarborough                         Rouge,Malvern  43.806686 -79.194353   
1  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535 -79.160497   
2  Scarborough       Guildwood,Morningside,West Hill  43.763573 -79.188711   
3  Scarborough                                Woburn  43.770992 -79.216917   
4  Scarborough                             Cedarbrae  43.773136 -79.239476   

  Postcode  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0      M1B               0  Fast Food Restaurant             Drugstore   
1      M1C               2                   Bar   Dumpling Restaurant   
2      M1E               2        Medical Center        Breakfast Spot   
3      M1G               2           Coffee Shop     Korean Restaurant   
4      M1H               2    Athletics & Sports   Fried Chicken Joint   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0    Dim Sum Restaurant                 Diner        Discount Store   
1                 Diner        Discount Store               Dog Run   
2   Rental Car Location    Mexican Restaurant          Intersection   
3      Insurance Office             Drugstore    Dim Sum Restaurant   
4      Hakka Restaurant       Thai Restaurant                Bakery   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0               Dog Run      Doner Restaurant            Donut Shop   
1      Doner Restaurant            Donut Shop             Drugstore   
2           Pizza Place                   Spa     Electronics Store   
3                 Diner        Discount Store               Dog Run   
4  Caribbean Restaurant                  Bank      Doner Restaurant   

         9th Most Common Venue 10th Most Common Venue  
0          Dumpling Restaurant            College Gym  
1                  Yoga Studio           Dessert Shop  
2  Eastern European Restaurant    Dumpling Restaurant  
3             Doner Restaurant             Donut Shop  
4                        Diner         Discount Store

Finally, let's visualize the resulting clusters

In [83]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(district_merged['Latitude'], district_merged['Longitude'], district_merged['Postcode'], district_merged['Cluster Labels'].astype(np.int64)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='menu5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [84]:
district_merged.loc[district_merged['Cluster Labels'] == 0, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

borough Postcode  Cluster Labels 1st Most Common Venue  \
0   Scarborough      M1B               0  Fast Food Restaurant   
14  Scarborough      M1V               0                  Park   
21   North York      M2P               0                  Bank   
23   North York      M3A               0                  Park   
28   North York      M3K               0                  Park   
72         York      M6E               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0              Drugstore    Dim Sum Restaurant                 Diner   
14            Playground           Coffee Shop           Yoga Studio   
21                  Park     Electronics Store           Yoga Studio   
23  Fast Food Restaurant                  Pool     Food & Drink Shop   
28               Airport           Snack Place           Yoga Studio   
72              Pharmacy         Women's Store  Fast Food Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Discount Store               Dog Run      Doner Restaurant   
14            Donut Shop          Dessert Shop    Dim Sum Restaurant   
21    Dim Sum Restaurant                 Diner        Discount Store   
23           Yoga Studio      Doner Restaurant    Dim Sum Restaurant   
28            Donut Shop    Dim Sum Restaurant                 Diner   
72                Market            Donut Shop    Dim Sum Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0             Donut Shop   Dumpling Restaurant            College Gym  
14                 Diner        Discount Store                Dog Run  
21               Dog Run      Doner Restaurant             Donut Shop  
23                 Diner        Discount Store                Dog Run  
28        Discount Store               Dog Run       Doner Restaurant  
72                 Diner        Discount Store                Dog Run

#### Cluster 2

In [85]:
district_merged.loc[district_merged['Cluster Labels'] == 1, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

borough  Cluster Labels 1st Most Common Venue  \
61  Central Toronto               1                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
61            Donut Shop          Dessert Shop    Dim Sum Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
61                 Diner        Discount Store               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
61      Doner Restaurant           Yoga Studio       Department Store

#### Cluster 3

In [86]:
district_merged.loc[district_merged['Cluster Labels'] == 2, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

borough  Cluster Labels   1st Most Common Venue  \
1        Scarborough               2                     Bar   
2        Scarborough               2          Medical Center   
3        Scarborough               2             Coffee Shop   
4        Scarborough               2      Athletics & Sports   
5        Scarborough               2       Convenience Store   
6        Scarborough               2        Department Store   
7        Scarborough               2                  Bakery   
8        Scarborough               2                   Motel   
9        Scarborough               2   General Entertainment   
10       Scarborough               2       Indian Restaurant   
11       Scarborough               2             Auto Garage   
12       Scarborough               2          Breakfast Spot   
13       Scarborough               2             Pizza Place   
15       Scarborough               2      Chinese Restaurant   
16        North York               2    Fast Food Restaurant   
17        North York               2          Clothing Store   
18        North York               2                    Bank   
20        North York               2             Coffee Shop   
22        North York               2                Pharmacy   
24        North York               2    Gym / Fitness Center   
25        North York               2        Asian Restaurant   
26        North York               2             Coffee Shop   
27        North York               2  Furniture / Home Store   
29        North York               2           Shopping Mall   
30        North York               2        Business Service   
31        North York               2    Gym / Fitness Center   
32        North York               2   Portuguese Restaurant   
33         East York               2    Fast Food Restaurant   
34         East York               2                Pharmacy   
35      East Toronto               2       Health Food Store   
..               ...             ...                     ...   
65  Downtown Toronto               2                    Café   
66  Downtown Toronto               2         Airport Service   
67  Downtown Toronto               2             Coffee Shop   
68  Downtown Toronto               2                    Café   
69        North York               2  Furniture / Home Store   
70        North York               2                     Pub   
71              York               2                   Field   
73  Downtown Toronto               2                    Café   
74      West Toronto               2                Pharmacy   
75      West Toronto               2                     Bar   
76      West Toronto               2                    Café   
77        North York               2                    Park   
78              York               2                     Bar   
79              York               2                Bus Line   
80      West Toronto               2                     Bar   
81      West Toronto               2          Breakfast Spot   
82      West Toronto               2                    Café   
83      Queen's Park               2             Coffee Shop   
84       Mississauga               2                   Hotel   
85      East Toronto               2      Light Rail Station   
86         Etobicoke               2               Pet Store   
87         Etobicoke               2             Pizza Place   
90         Etobicoke               2           Grocery Store   
91         Etobicoke               2                    Bank   
92         Etobicoke               2                Pharmacy   
93        North York               2             Pizza Place   
96         Etobicoke               2             Pizza Place   
97         Etobicoke               2             Pizza Place   
98         Etobicoke               2           Grocery Store   
99         Etobicoke               2               Drugstore   

            2nd Most Common Venue       3rd 

#### Cluster 4

In [87]:
district_merged.loc[district_merged['Cluster Labels'] == 3, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

borough  Cluster Labels 1st Most Common Venue  \
19        North York               3             Cafeteria   
38         East York               3                  Park   
48  Downtown Toronto               3                  Park   
88         Etobicoke               3                  Park   
95              York               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
19                  Park            Donut Shop          Dessert Shop   
38     Convenience Store         Metro Station    Dim Sum Restaurant   
48            Playground                 Trail           Yoga Studio   
88                 River           Yoga Studio      Doner Restaurant   
95           Yoga Studio             Drugstore    Dim Sum Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
19    Dim Sum Restaurant                 Diner        Discount Store   
38                 Diner        Discount Store               Dog Run   
48      Doner Restaurant      Department Store          Dessert Shop   
88          Dessert Shop    Dim Sum Restaurant                 Diner   
95                 Diner        Discount Store               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
19               Dog Run      Doner Restaurant              Drugstore  
38      Doner Restaurant            Donut Shop            Yoga Studio  
48    Dim Sum Restaurant                 Diner         Discount Store  
88        Discount Store               Dog Run             Donut Shop  
95      Doner Restaurant            Donut Shop    Dumpling Restaurant

#### Cluster 5

In [88]:
district_merged.loc[district_merged['Cluster Labels'] == 4, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

borough  Cluster Labels       1st Most Common Venue  \
89   Etobicoke               4              Baseball Field   
94  North York               4  Construction & Landscaping   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
89           Yoga Studio   Dumpling Restaurant                 Diner   
94        Baseball Field           Yoga Studio   Dumpling Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
89        Discount Store               Dog Run      Doner Restaurant   
94                 Diner        Discount Store               Dog Run   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
89            Donut Shop             Drugstore  Eastern European Restaurant  
94      Doner Restaurant            Donut Shop                    Drugstore